In [8]:
## Convertir datos crudos e inconsistentes en un dataset confiable, estructurado y reutilizable para análisis, Spark y dashboards.

import pandas as pd
import os

##########################################################
# Mostramos las tablas con todas sus columnas seguidas
pd.set_option('display.max_columns', None)      # Mostrar todas las columnas
pd.set_option('display.max_colwidth', None)     # Sin límite en ancho de columna
pd.set_option('display.expand_frame_repr', False)  # No dividir el dataframe en varias líneas
pd.set_option('display.width', 0)               # Ajusta el ancho a la ventana del terminal
##########################################################

# Cargar dataset
# El dataset raw se considera inmutable. Todas las transformaciones se realizan sobre copias para garantizar trazabilidad.
df = pd.read_csv('../data/raw/global_earthquakes_10y.csv')

print(df.head(20))

                                time  latitude   longitude   depth   mag magType   nst    gap     dmin   rms net          id                   updated                                  place        type  horizontalError  depthError  magError  magNst    status locationSource magSource
0   2015-12-30 23:20:56.840000+00:00    0.0229  123.819400  143.05  4.60      mb   NaN   51.0   1.4880  0.83  us  us10004aif  2016-03-18T01:13:08.040Z      101 km SE of Gorontalo, Indonesia  earthquake             6.40        6.70     0.083    43.0  reviewed             us        us
1   2015-12-30 21:29:23.040000+00:00  -30.6777  -71.734600   25.62  4.50      mb   NaN  180.0   0.0850  0.94  us  us10004ahx  2016-03-18T01:13:08.040Z               52 km W of Ovalle, Chile  earthquake             3.90        5.40     0.192     8.0  reviewed             us        us
2   2015-12-30 19:50:48.210000+00:00  -40.5641  173.421900  142.40  4.60      mb   NaN   65.0   0.6530  0.70  us  us10004agf  2022-08-02T02:00:26.95

In [9]:
## Se priorizan variables relevantes para análisis sísmico y visualización. Métricas técnicas con alto porcentaje de nulos se excluyen para simplificar el modelo.

df = df[['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'place', 'status']]  
print(df.head(20))

                                time  latitude   longitude   depth   mag magType                                  place    status
0   2015-12-30 23:20:56.840000+00:00    0.0229  123.819400  143.05  4.60      mb      101 km SE of Gorontalo, Indonesia  reviewed
1   2015-12-30 21:29:23.040000+00:00  -30.6777  -71.734600   25.62  4.50      mb               52 km W of Ovalle, Chile  reviewed
2   2015-12-30 19:50:48.210000+00:00  -40.5641  173.421900  142.40  4.60      mb       61 km ENE of Takaka, New Zealand  reviewed
3   2015-12-30 18:26:34.610000+00:00  -24.4715 -175.871600   24.21  4.90      mb                         south of Tonga  reviewed
4   2015-12-30 14:46:47.730000+00:00   -5.9568  154.961100  184.56  4.50      mb  70 km NW of Panguna, Papua New Guinea  reviewed
5   2015-12-30 12:57:36.210000+00:00   33.8836  137.321100  338.23  4.50      mb               79 km SSE of Toba, Japan  reviewed
6   2015-12-30 12:50:12.930000+00:00  -18.0505 -178.708500  607.86  4.50      mb          

In [10]:
## Normalización de tipos es clave para evitar errores posteriores en agregaciones y procesamiento distribuido.
df['time'] = pd.to_datetime(df['time'], errors='coerce')    
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df['mag'] = pd.to_numeric(df['mag'], errors='coerce')   
print(df.dtypes)

time         datetime64[ns, UTC]
latitude                 float64
longitude                float64
depth                    float64
mag                      float64
magType                   object
place                     object
status                    object
dtype: object


In [12]:
## crear columnas year y month
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month   
print(df.head(20))


                               time  latitude   longitude   depth   mag magType                                  place    status  year  month
0  2015-12-30 23:20:56.840000+00:00    0.0229  123.819400  143.05  4.60      mb      101 km SE of Gorontalo, Indonesia  reviewed  2015     12
1  2015-12-30 21:29:23.040000+00:00  -30.6777  -71.734600   25.62  4.50      mb               52 km W of Ovalle, Chile  reviewed  2015     12
2  2015-12-30 19:50:48.210000+00:00  -40.5641  173.421900  142.40  4.60      mb       61 km ENE of Takaka, New Zealand  reviewed  2015     12
3  2015-12-30 18:26:34.610000+00:00  -24.4715 -175.871600   24.21  4.90      mb                         south of Tonga  reviewed  2015     12
4  2015-12-30 14:46:47.730000+00:00   -5.9568  154.961100  184.56  4.50      mb  70 km NW of Panguna, Papua New Guinea  reviewed  2015     12
5  2015-12-30 12:57:36.210000+00:00   33.8836  137.321100  338.23  4.50      mb               79 km SSE of Toba, Japan  reviewed  2015     12
6  201

In [ ]:
## La categorización de magnitud facilita análisis agregados y visualización para usuarios no técnicos.
def categorize_magnitude(mag):
    if mag < 4.0:
        return 'Low'
    elif 4.0 <= mag < 5.0:
        return 'Moderate'
    elif 5.0 <= mag < 6.0:
        return 'Strong'
    else:
        return 'Major'
df['magnitude_category'] = df['mag'].apply(categorize_magnitude)
print(df.head(20))





                               time  latitude   longitude   depth   mag magType                                  place    status  year  month magnitude_category
0  2015-12-30 23:20:56.840000+00:00    0.0229  123.819400  143.05  4.60      mb      101 km SE of Gorontalo, Indonesia  reviewed  2015     12           Moderate
1  2015-12-30 21:29:23.040000+00:00  -30.6777  -71.734600   25.62  4.50      mb               52 km W of Ovalle, Chile  reviewed  2015     12           Moderate
2  2015-12-30 19:50:48.210000+00:00  -40.5641  173.421900  142.40  4.60      mb       61 km ENE of Takaka, New Zealand  reviewed  2015     12           Moderate
3  2015-12-30 18:26:34.610000+00:00  -24.4715 -175.871600   24.21  4.90      mb                         south of Tonga  reviewed  2015     12           Moderate
4  2015-12-30 14:46:47.730000+00:00   -5.9568  154.961100  184.56  4.50      mb  70 km NW of Panguna, Papua New Guinea  reviewed  2015     12           Moderate
5  2015-12-30 12:57:36.210000+00:0

In [ ]:
# Guardar dataset limpio
# El dataset procesado se guarda separado del raw para facilitar reproducibilidad y downstream processing.
output_path = '../data/processed/earthquakes_clean.csv'
df.to_csv(output_path, index=False)
print(f'Dataset limpio guardado en: {os.path.abspath(output_path)}')

Dataset limpio guardado en: c:\Users\FLOREN\Desktop\PythonGitHub\CSV_ETL_Python_Spark_PowerBI\data\processed\earthquakes_clean.csv
